In [1]:
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly

net = pp.create_empty_network() # creating village network

In [2]:
# Defining parameters
# # Voltage data
# v_h_kv = 132
# v_m_kv = 11
# v_l_kv = 0.415

# # Demand / supply parameters
# ADMD_MW = 2e-3
# pv_MWp = 3.4e-3

In [3]:
# net = pp.create_empty_network()

# b = {}

# # Slack Bus
# b[0] = pp.create_bus(net, vn_kv = v_h_kv, name = f"Bus 0", geodata = (0,0))
# pp.create_ext_grid(net, bus=b[0], vm_pu=1.00, name = "Slack bus")

# # 11kV bus asfter first step down
# b[1] = pp.create_bus(net, vn_kv = v_m_kv, name = f"Bus 1", geodata = (1,1))

# # Four 0.415kV buses for
# for i in range(2,6):
#     b[i] = pp.create_bus(net, vn_kv = v_l_kv, name = f"Bus 2", geodata = (i, i))

# # three buses at 0.415kV, eight houses in each bus at the end of a 0.3km run for a total power of 16kW 
# for i in range(6,9):
#     b[i] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus 2", geodata = (i, i))
#     pp.create_ext_grid(net, bus=b[i oxfor], vm_pu=1.00, name = "Slack bus")



In [4]:
#132/11 trafo

# pp.create_transformer_from_parameters(net, hv_bus=b[0], lv_bus=b[1], sn_mva=0.5, vn_hv_kv=v_h_kv, vn_lv_kv=v_m_kv, vkr_percent=10, vk_percent=1, pfe_kw=5, i0_percent=5)



In [5]:
'''
Zooming in to the very last group of 24 houses served by one 100kVA 11/0.415 transformer 
Taking the 415V output as the slack bus that serves three groups of houses at 16kW each 


'''

p_mw = 0.016
q_mvar = 0.007749

net_mini =  pp.create_empty_network()

b = {}
# Slack Bus
b[0] = pp.create_bus(net_mini, vn_kv = 0.415, name = f"Bus 0", geodata = (0,0))
pp.create_ext_grid(net_mini, bus=b[0], vm_pu=0.95, name = "Slack bus")

#three load buses of 16kW each
for i in range(1,7):
    b[i] = pp.create_bus(net_mini, vn_kv = 0.415, name = f"Bus {i}")
    pp.create_load(net_mini, bus=b[i], p_mw=p_mw, q_mvar=q_mvar, name=f"load_{i}")  
    pp.create_line_from_parameters(net_mini, from_bus=b[0], to_bus=b[i],length_km=0.4, r_ohm_per_km =1.6, x_ohm_per_km = 0.135, c_nf_per_km = 0.5, max_i_ka=0.05, name=f"line_{str(i)}")
    

In [6]:
net_mini.bus

,name,vn_kv,type,zone,in_service
0,Bus 0,0.415,b,None,True
1,Bus 1,0.415,b,None,True
2,Bus 2,0.415,b,None,True
3,Bus 3,0.415,b,None,True
4,Bus 4,0.415,b,None,True
5,Bus 5,0.415,b,None,True
6,Bus 6,0.415,b,None,True


In [7]:
pp.runpp(net_mini, algorithm="gs");
net_mini.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949
1,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949
2,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949
3,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949
4,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949
5,0.017519,0.007877,-0.016,-0.007749,0.001519,0.000128,0.02813,0.02813,0.02813,0.95,0.0,0.879227,1.631368,56.25949


In [8]:
net_mini.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.950000,0.000000,-0.105116,-0.047263
1,0.879227,1.631368,0.016000,0.007749
2,0.879227,1.631368,0.016000,0.007749
3,0.879227,1.631368,0.016000,0.007749
4,0.879227,1.631368,0.016000,0.007749
5,0.879227,1.631368,0.016000,0.007749
6,0.879227,1.631368,0.016000,0.007749


In [9]:
'''
One stretch of 8 houses, 2kW each 

'''

p_mw_small = 0.002
q_mvar_small = 968.6e-6

net_micro =  pp.create_empty_network()

b = {}
# Slack Bus
b[0] = pp.create_bus(net_micro, vn_kv = 0.415, name = f"Bus 0", geodata = (0,0))
pp.create_ext_grid(net_micro, bus=b[0], vm_pu=0.95, name = "Slack bus")

#eight load buses of 2kW each
for i in range(1,9):
    b[i] = pp.create_bus(net_micro, vn_kv = 0.415, name = f"Bus {i}")
    pp.create_load(net_micro, bus=b[i], p_mw=p_mw_small, q_mvar=q_mvar_small, name=f"load_{i}")  
for i in range(0,8):
    pp.create_line_from_parameters(net_micro, from_bus=b[i], to_bus=b[i+1],length_km=0.05, r_ohm_per_km =1.6, x_ohm_per_km = 0.135, c_nf_per_km = 0.5, max_i_ka=0.05, name=f"line_{str(i)}")



In [10]:
pp.runpp(net_micro, algorithm="gs");
net_micro.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.016554,0.007795,-0.016381,-0.007781,0.000172,1.453777e-05,0.026795,0.026795,0.026795,0.950000,0.000000,0.941590,0.190383,53.590082
1,0.014381,0.006812,-0.014249,-0.006801,0.000133,1.119314e-05,0.023512,0.023512,0.023512,0.941590,0.190383,0.934216,0.359783,47.023670
2,0.012249,0.005833,-0.012151,-0.005824,0.000098,8.263821e-06,0.020203,0.020203,0.020203,0.934216,0.359783,0.927884,0.507128,40.405340
3,0.010151,0.004856,-0.010082,-0.004850,0.000068,5.762541e-06,0.016871,0.016871,0.016871,0.927884,0.507128,0.922599,0.631461,33.741791
4,0.008082,0.003881,-0.008038,-0.003878,0.000044,3.700371e-06,0.013520,0.013520,0.013520,0.922599,0.631461,0.918366,0.731958,27.040043
5,0.006038,0.002909,-0.006014,-0.002907,0.000025,2.086588e-06,0.010154,0.010154,0.010154,0.918366,0.731958,0.915189,0.807942,20.307391
6,0.004014,0.001938,-0.004003,-0.001937,0.000011,9.285431e-07,0.006776,0.006776,0.006776,0.915189,0.807942,0.913069,0.858891,13.551355
7,0.002003,0.000969,-0.002000,-0.000969,0.000003,2.315633e-07,0.003390,0.003390,0.003390,0.913069,0.858891,0.912009,0.884455,6.779631


In [11]:
net_micro.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.950000,0.000000,-0.016554,-0.007795
1,0.941590,0.190383,0.002000,0.000969
2,0.934216,0.359783,0.002000,0.000969
3,0.927884,0.507128,0.002000,0.000969
4,0.922599,0.631461,0.002000,0.000969
5,0.918366,0.731958,0.002000,0.000969
6,0.915189,0.807942,0.002000,0.000969
7,0.913069,0.858891,0.002000,0.000969
8,0.912009,0.884455,0.002000,0.000969
